ANLP Assignment 1

Saurabh Hebbalkar

19232697

In [0]:
#Required Imports
%tensorflow_version 2.x 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils import shuffle
from collections import Counter
import tensorflow as tf
from tensorflow.keras import layers
from keras.layers import Embedding
from sklearn import svm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
import itertools
from sklearn import metrics
import nltk
nltk.download('stopwords')
stop = stopwords.words('english')

print(tf.__version__)

In [0]:
#loading the File
from google.colab import files
files.upload()

In [0]:
#Loding the file and storing in the dataframe
data = pd.read_csv('SemEval2018-T3-train-taskA.txt', sep='\t')
#Renaming the columns
data.columns = ['Tweet_index', 'Label','Tweet_text']

#Storing the Tweet Text as list
corpus = data['Tweet_text'].tolist()
#Storing the lables as sentiments
sentiment = data['Label']

positive_example = 0
negative_example = 0

#Getting all the words in one list
wordList =[]
for words in corpus:
    wordList.append(words.lower().strip().split())

wordList = list(itertools.chain.from_iterable(wordList))
#Set fetches the unique words and we add it to the vocabulary
vocabulary = set(wordList)
vocabulary_len = len(vocabulary)

for i in range(len(sentiment)):
  if(sentiment[i] == 1):
    positive_example = positive_example + 1
  else:
    negative_example = negative_example + 1

print("Size of the Vocabulary:",vocabulary_len)
print("Total Number of Positive Examples:",positive_example)
print("Total Number of Negative Examples:",negative_example)

Task 2


In [0]:
#Using the data for train test split
x_data = data['Tweet_text']
y_labels = data['Label']


In [0]:
#Function to calculate precision recall and F-Measure

def calculate_metrics(y_true, y_pred):
  #Accepting Actual values and Predicted values to calculate metric scores
  precision_score = metrics.precision_score(y_true, y_pred)
  recall_score = metrics.recall_score(y_true, y_pred)
  f1_score = metrics.f1_score(y_true, y_pred)
  return precision_score, recall_score, f1_score

Task 3

https://towardsdatascience.com/sarcasm-detection-step-towards-sentiment-analysis-84cb013bb6db

In [0]:
#Converting words into tokens(numbers)
vectorizer = CountVectorizer(ngram_range=(1,1))
vectorizer.fit(corpus)
#transforming output to array
x_data = vectorizer.transform(corpus).toarray()
#Splitting train test data
#I am splliting 30% for test and 70% for training
X_train, X_test, y_train, y_test = train_test_split(x_data,y_labels,test_size=0.3)


In [0]:
# Logistic Regression has the log linear model

lr = LogisticRegression()
#Training the model on train data
lr.fit(X_train, y_train)
#Creating list for storing the predictions
prediction = []
for i in X_test:
  #predicting all the test data
  prediction.append(lr.predict(i.reshape(1,-1)))

#calling the defined metrics function to get the Precison recall and F-measure score
print("Precison, Recall and F-Measure Score for Logistic Regression is:")
print(calculate_metrics(y_test, prediction))

Task 4


In [0]:
#Max Lenght of the Output Dimension
max_len = 128

#Getting the data for Test Train split
x_data_nn = data['Tweet_text'].tolist()
y_labels_nn = data['Label'].tolist()

#Splitting the data in to test and train with 30% for test and 70% for train
X_train_nn, X_test_nn, y_train_nn, y_test_nn = train_test_split(x_data_nn,y_labels_nn,test_size=0.3, shuffle=True)
print(len(X_train_nn), len(X_test_nn))


In [0]:
#Preprocessing data for Keras Model
#Converting class vector to binary class matrix.
y_train_nn = to_categorical(y_train_nn)

#Converting sentences into tokens as an input for the model
tokenizer = Tokenizer(num_words = vocabulary_len)

#Vectorizing the corpus
#Updating internal vocabulary based on a list of texts
tokenizer.fit_on_texts(X_train_nn)

#Storing Label test data before preprocessing into another list so that it can be used later with original format
y_test_nn1 = y_test_nn

#Transforming each text in texts to a sequence of integers
X_train_nn = tokenizer.texts_to_sequences(X_train_nn)

#Padding sequences to the same length. The smaller sententces will be padded with 0 and thus model understands it
X_train_nn = pad_sequences(X_train_nn, maxlen = max_len)

#Transforming each text in texts to a sequence of integers for test data
X_test_nn = tokenizer.texts_to_sequences(X_test_nn)

#Padding sequences to the same length for test data
X_test_nn = pad_sequences(X_test_nn, maxlen = max_len)
y_test_nn = to_categorical(y_test_nn)

In [0]:
#Keras Model
#Sequential Model
#Embedding layer for similarity in words
#This layer encodes the input sequence into a sequence of dense vectors.
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocabulary_len, max_len),
])
#Implementing LSTM layer
#LSTM layer uses gates to control the memorizing process
#It transforms the vector sequence into a single vector of size comtaining sequence information.
model.add(tf.keras.layers.LSTM(100))

#Dense layer with activation function as softmax
model.add(tf.keras.layers.Dense(2, activation='softmax'))

#Compiling the model with binary crossentropy loss functions and stochastic gradient descent optimizer
model.compile(loss = 'binary_crossentropy', optimizer='sgd', metrics = ['accuracy'])

#Summerizing the model
model.summary()

In [0]:
#Traning the model with 10 batch size and 10 iterations
model.fit(x =X_train_nn, y=y_train_nn ,epochs = 10, batch_size = 10,
                    validation_data=(X_test_nn, y_test_nn)
                    )

In [0]:
#Saving the Model
model.save('FinalModel.h5')

In [0]:
#Predicting the classes for the test data and storing in the list
predictionRNN = model.predict_classes(X_test_nn)

#Fecthing the Metric results
print("Precison, Recall and F-Measure Score is:")
print(calculate_metrics(y_test_nn1,predictionRNN))

Task 5


Improved Model

---


From the above observation, the (Task 4) Keras model was overfitting. The model was performing decent on train data but was performing poorly on test data. The metric score was also poor. The reason for this poor performace was the pre-processing of data and the structure of the model. The preprocessing was only converting string into lowercase and then generating tokens. The model had Stochastic Gradient Descent optimizer and did not have dropout layer.
In the imporoved model I added  Dropout layer as it randomly ignores the nodes and thus avoides overfitting. As per the lecture 3; hinge loss works well for the binary classifcation and so I used the hing loss function in the model for better results. Adaptive optimizer "adam" is used instead of SGD. I also preprocessed the data thoroughly and removed the stopwords, punctuations and special characters. I also removed numbers from the data as it might create problem in tokenzing the input for the keras model.
The model performance did not improve my much but the improvment is visible through evaluation metric.

In [0]:
#Data Enhancement 

data = pd.read_csv('SemEval2018-T3-train-taskA.txt', sep='\t')
data.columns = ['Tweet_index', 'Label','Tweet_text']

#Data preprocessing and removing the unwanted data
#Removing Numbers
data['Tweet_text'] = data['Tweet_text'].str.replace('\d+', '')
#Removing -- character
data['Tweet_text'] = data['Tweet_text'].str.replace("--", '')
#Removing Punctuations
data['Tweet_text'] = data['Tweet_text'].str.replace('[^\w\s]','')
#Lowercase and splitting sentences into tokens
data['Tweet_text'] = data['Tweet_text'].str.lower()
#Removing Stop words
data['Tweet_text'].apply(lambda x: [item for item in x if item not in stop])

corpus = data['Tweet_text'].tolist()

wordList =[]
for words in corpus:
      #print(words)
      wordList.extend(words.split())

#wordList = list(itertools.chain.from_iterable(wordList))
vocabulary = set(wordList)
vocabulary_len = len(vocabulary)

#After preprocessing the data, the updated vacabluary has less words
print("Size of the Updated Vocabulary:",vocabulary_len)


In [0]:
x_data_nn = data['Tweet_text'].tolist()
y_labels = data['Label'].tolist()

#Max Lenght of the Output Dimension
max_len = 128

#Getting the data for Test Train split
x_data_nn = data['Tweet_text'].tolist()
y_labels_nn = data['Label'].tolist()

#Splitting the data in to test and train with 30% for test and 70% for train
X_train_nn, X_test_nn, y_train_nn, y_test_nn = train_test_split(x_data_nn,y_labels_nn,test_size=0.3, shuffle=True)
print(len(X_train_nn), len(X_test_nn))

#Preprocessing data for Keras Model
#Converting class vector to binary class matrix.
y_train_nn = to_categorical(y_train_nn)

#Converting sentences into tokens as an input for the model
tokenizer = Tokenizer(num_words = vocabulary_len)

#Vectorizing the corpus
#Updating internal vocabulary based on a list of texts
tokenizer.fit_on_texts(X_train_nn)

#Storing Label test data before preprocessing into another list so that it can be used later with original format
y_test_nn1 = y_test_nn

#Transforming each text in texts to a sequence of integers
X_train_nn = tokenizer.texts_to_sequences(X_train_nn)

#Padding sequences to the same length. The smaller sententces will be padded with 0 and thus model understands it
X_train_nn = pad_sequences(X_train_nn, maxlen = max_len)

#Transforming each text in texts to a sequence of integers for test data
X_test_nn = tokenizer.texts_to_sequences(X_test_nn)

#Padding sequences to the same length for test data
X_test_nn = pad_sequences(X_test_nn, maxlen = max_len)
y_test_nn = to_categorical(y_test_nn)


In [0]:
#Sequential Model with Embedding layer just like above
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocabulary_len, max_len),
])
#Adding two LSTM layers one returns the sequence other doesnt
model.add(tf.keras.layers.LSTM(100, return_sequences=True))
model.add(tf.keras.layers.LSTM(100, return_sequences=False))
#Drouput layer is used to avoid overfitting. Randomly ignores the nodes from the layer and thus overfitting is avoided
model.add(tf.keras.layers.Dropout(0.4))
#Activation function used is softmax
model.add(tf.keras.layers.Dense(2, activation='relu'))

#Using Hinge loss as it is better for binary classification (From lecture 3 slides) & Adaptive Adam optimizer
model.compile(loss = 'squared_hinge', optimizer=tf.keras.optimizers.Adam(1e-4), metrics = ['accuracy'])

#Summary of Model
model.summary()

In [0]:
#Model Training
model.fit(x =X_train_nn, y=y_train_nn ,epochs = 10, batch_size = 10,
                    validation_data=(X_test_nn, y_test_nn)
                    )

In [0]:
ENpredictionRNN = model.predict_classes(X_test_nn)

print(calculate_metrics(y_test_nn1,ENpredictionRNN))
#predictionsvm.append(svclassifier.predict(i.reshape(1,-1)))